In [ ]:
pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 20.6 MB/s eta 0:00:00


In [ ]:
# ============================================================
# ANEXO B — Sistema de inferencia difusa (Mamdani) para riesgo de lixiviación
# ============================================================
# Contenido:
#   B.1) Sistema difuso conceptual (genérico): membresías + reglas + superficies
#   B.2) Aplicación a datos reales (USCO): Impacto_fuzzy + Riesgo_fuzzy + gráficas + export
#
# Requisitos:
#   pip install numpy pandas matplotlib scikit-fuzzy
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl


# ------------------------------------------------------------
# B.1) DEFINICIÓN DEL SISTEMA DIFUSO (CONCEPTUAL / GENÉRICO)
# ------------------------------------------------------------

# Rangos (universos) para cada variable
N_min, N_max = 0, 15          # mg/L NO3–N
MO_min, MO_max = 0, 6         # %
DA_min, DA_max = 0.9, 1.9     # g/cm3
CE_min, CE_max = 0, 2000      # µS/cm
I_min, I_max = 0, 100         # índice 0–100

# Definición de variables difusas
N  = ctrl.Antecedent(np.linspace(N_min,  N_max,  401), 'N_agua_NO3N')
MO = ctrl.Antecedent(np.linspace(MO_min, MO_max, 401), 'MO_suelo')
DA = ctrl.Antecedent(np.linspace(DA_min, DA_max, 401), 'DA_suelo')
CE = ctrl.Antecedent(np.linspace(CE_min, CE_max, 401), 'CE_agua')
Impacto = ctrl.Consequent(np.linspace(I_min, I_max, 401), 'Impacto')

# -------------------------
# Funciones de membresía
# -------------------------
# Materia orgánica (MO): baja (mayor riesgo), media, alta (menor riesgo)
MO['baja']  = fuzz.trapmf(MO.universe, [0, 0, 1.10, 1.60])
MO['media'] = fuzz.trimf(MO.universe, [1.50, 2.25, 3.00])
MO['alta']  = fuzz.trapmf(MO.universe, [2.90, 3.40, 6.00, 6.00])

# Nitratos en agua como NO3–N
N['bajo']  = fuzz.trapmf(N.universe, [0, 0, 7.0, 10.0])
N['medio'] = fuzz.trimf(N.universe, [8.0, 10.5, 13.0])
N['alto']  = fuzz.trapmf(N.universe, [11.0, 13.0, 15.0, 15.0])

# Densidad aparente en suelos arroceros
DA['baja']  = fuzz.trapmf(DA.universe, [DA_min, DA_min, 1.10, 1.25])
DA['media'] = fuzz.trimf(DA.universe, [1.20, 1.35, 1.50])
DA['alta']  = fuzz.trapmf(DA.universe, [1.55, 1.65, DA_max, DA_max])

# Conductividad eléctrica del agua (µS/cm)
CE['baja']  = fuzz.trapmf(CE.universe, [0, 0, 600, 850])
CE['media'] = fuzz.trimf(CE.universe, [800, 1150, 1500])
CE['alta']  = fuzz.trapmf(CE.universe, [1400, 1600, 2000, 2000])

# Salida: Impacto ambiental (0–100)
Impacto['bajo']  = fuzz.trapmf(Impacto.universe, [0, 0, 25, 40])
Impacto['medio'] = fuzz.trimf(Impacto.universe, [30, 50, 70])
Impacto['alto']  = fuzz.trapmf(Impacto.universe, [60, 75, 100, 100])

# -------------------------
# Base de reglas difusas
# -------------------------
rules = [
    # N alto: domina el riesgo, modulado por MO y DA
    ctrl.Rule(N['alto'] & MO['baja'] & DA['baja'],  Impacto['alto']),
    ctrl.Rule(N['alto'] & MO['baja'] & DA['media'], Impacto['alto']),
    ctrl.Rule(N['alto'] & MO['baja'] & DA['alta'],  Impacto['alto']),
    ctrl.Rule(N['alto'] & MO['media'] & DA['baja'], Impacto['alto']),
    ctrl.Rule(N['alto'] & MO['media'] & DA['media'],Impacto['alto']),
    ctrl.Rule(N['alto'] & MO['media'] & DA['alta'], Impacto['medio']),
    ctrl.Rule(N['alto'] & MO['alta'] & DA['baja'],  Impacto['medio']),
    ctrl.Rule(N['alto'] & MO['alta'] & DA['media'], Impacto['medio']),
    ctrl.Rule(N['alto'] & MO['alta'] & DA['alta'],  Impacto['medio']),

    # N medio
    ctrl.Rule(N['medio'] & MO['baja'] & DA['baja'],  Impacto['alto']),
    ctrl.Rule(N['medio'] & MO['baja'] & DA['media'], Impacto['medio']),
    ctrl.Rule(N['medio'] & MO['baja'] & DA['alta'],  Impacto['medio']),
    ctrl.Rule(N['medio'] & MO['media'] & DA['baja'], Impacto['medio']),
    ctrl.Rule(N['medio'] & MO['media'] & DA['media'],Impacto['medio']),
    ctrl.Rule(N['medio'] & MO['media'] & DA['alta'], Impacto['bajo']),
    ctrl.Rule(N['medio'] & MO['alta'] & DA['baja'],  Impacto['medio']),
    ctrl.Rule(N['medio'] & MO['alta'] & DA['media'], Impacto['bajo']),
    ctrl.Rule(N['medio'] & MO['alta'] & DA['alta'],  Impacto['bajo']),

    # N bajo
    ctrl.Rule(N['bajo'] & MO['baja'] & DA['baja'],  Impacto['medio']),
    ctrl.Rule(N['bajo'] & MO['baja'] & DA['media'], Impacto['medio']),
    ctrl.Rule(N['bajo'] & MO['baja'] & DA['alta'],  Impacto['bajo']),
    ctrl.Rule(N['bajo'] & MO['media'] & DA['baja'], Impacto['bajo']),
    ctrl.Rule(N['bajo'] & MO['media'] & DA['media'],Impacto['bajo']),
    ctrl.Rule(N['bajo'] & MO['media'] & DA['alta'], Impacto['bajo']),
    ctrl.Rule(N['bajo'] & MO['alta'] & DA['alta'],  Impacto['bajo']),

    # Modulación por CE alta
    ctrl.Rule(CE['alta'] & N['alto'], Impacto['alto']),
    ctrl.Rule(CE['alta'] & N['medio'] & MO['baja'], Impacto['alto']),
    ctrl.Rule(CE['alta'] & N['medio'] & MO['media'], Impacto['medio']),
    ctrl.Rule(CE['alta'] & N['bajo'] & MO['baja'], Impacto['medio']),

    # Regla de cobertura (estabilidad)
    ctrl.Rule(N['bajo'] | N['medio'] | N['alto'], Impacto['medio']),
]

system = ctrl.ControlSystem(rules)


def fuzzy_impact(n, mo, da, ce):
    """Evalúa el sistema difuso y retorna Impacto (0–100)."""
    sim = ctrl.ControlSystemSimulation(system)
    sim.input["N_agua_NO3N"] = float(n)
    sim.input["MO_suelo"] = float(mo)
    sim.input["DA_suelo"] = float(da)
    sim.input["CE_agua"] = float(ce)
    sim.compute()
    return float(sim.output["Impacto"])


# ------------------------------------------------------------
# B.1.1) Figuras del sistema conceptual (membresías/superficies)
# ------------------------------------------------------------
def plot_memberships():
    """Grafica funciones de membresía para cada variable."""
    def _plot(var, title):
        plt.figure(figsize=(8.5, 4.8))
        for term_name, mf in var.terms.items():
            plt.plot(var.universe, mf.mf, label=term_name)
        plt.title(title, weight="bold")
        plt.xlabel(var.label)
        plt.ylabel("Grado de membresía")
        plt.grid(alpha=0.25)
        plt.legend()
        plt.tight_layout()
        plt.show()

    _plot(MO, "Funciones de membresía: MO_suelo (%)")
    _plot(DA, "Funciones de membresía: DA_suelo (g/cm³)")
    _plot(N,  "Funciones de membresía: N_agua_NO3N (mg/L)")
    _plot(CE, "Funciones de membresía: CE_agua (µS/cm)")
    _plot(Impacto, "Funciones de membresía: Impacto ambiental (0–100)")


def plot_surface_N_MO(da_const=1.35, ce_const=900, n_points=45):
    """
    Superficie conceptual: Impacto vs (N_agua_NO3N, MO_suelo),
    manteniendo DA y CE constantes.
    """
    Ns = np.linspace(N_min, N_max, n_points)
    MOs = np.linspace(MO_min, MO_max, n_points)
    Z = np.zeros((n_points, n_points))

    for i, n in enumerate(Ns):
        for j, mo in enumerate(MOs):
            Z[j, i] = fuzzy_impact(n=n, mo=mo, da=da_const, ce=ce_const)

    Xg, Yg = np.meshgrid(Ns, MOs)

    fig = plt.figure(figsize=(9, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(Xg, Yg, Z, linewidth=0, antialiased=True)
    ax.set_title("Superficie difusa: Impacto vs N_agua_NO3N y MO_suelo", weight="bold")
    ax.set_xlabel("N_agua_NO3N (mg/L)")
    ax.set_ylabel("MO_suelo (%)")
    ax.set_zlabel("Impacto (0–100)")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# B.2) APLICACIÓN A DATOS REALES (USCO)
# ------------------------------------------------------------
USCO_URL = "https://raw.githubusercontent.com/luicamongi/Impacto_ambiental/refs/heads/main/Data_suelo_USCO_agua_magdalena.csv?token=GHSAT0AAAAAADSELRPHBDWFE7225MBQGVPI2KXBE6A"

def run_usco_dataset(url_csv):
    """
    Carga el CSV USCO, ajusta unidades y calcula Impacto_fuzzy y Riesgo_fuzzy.
    IMPORTANTE: En este dataset la CE_agua está reportada en dS/m -> se convierte a µS/cm.
    """

    df = pd.read_csv(url_csv, encoding="utf-8-sig")
    df.columns = [c.strip() for c in df.columns]
    df = df.loc[:, ~df.columns.str.contains("^Unnamed", na=False)]

    # Columnas esperadas en tu dataset USCO (ajusta si cambian)
    col_mo = "M.O %"
    col_da = "D.A"
    col_nagua = "Nitrogeno agua mg/L"
    col_ce_dsm = "Conductividad Eléctrica agua dS/m"

    # Conversión a numérico
    for c in [col_mo, col_da, col_nagua, col_ce_dsm]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    work = df.copy()
    work["MO_suelo"] = work[col_mo]
    work["DA_suelo"] = work[col_da]
    work["N_agua_NO3N"] = work[col_nagua]

    # Conversión: 1 dS/m = 1000 µS/cm
    work["CE_agua"] = work[col_ce_dsm] * 1000.0

    # Filtrar filas completas
    work = work.dropna(subset=["MO_suelo", "DA_suelo", "N_agua_NO3N", "CE_agua"]).reset_index(drop=True)

    # Cálculo del impacto
    work["Impacto_fuzzy"] = work.apply(
        lambda r: fuzzy_impact(
            n=r["N_agua_NO3N"],
            mo=r["MO_suelo"],
            da=r["DA_suelo"],
            ce=r["CE_agua"]
        ),
        axis=1
    )

    # Clasificación cualitativa de riesgo
    def clasificar(x):
        if x < 40:
            return "Bajo"
        elif x < 70:
            return "Medio"
        return "Alto"

    work["Riesgo_fuzzy"] = work["Impacto_fuzzy"].apply(clasificar)
    return work


def plot_usco_results(work):
    """Genera gráficos de resultados para el dataset USCO (para reproducibilidad)."""

    # Histograma
    plt.figure(figsize=(8.5, 4.8))
    plt.hist(work["Impacto_fuzzy"], bins=12)
    plt.title("Distribución del Impacto_fuzzy (0–100)", weight="bold")
    plt.xlabel("Impacto_fuzzy")
    plt.ylabel("Frecuencia")
    plt.grid(alpha=0.25)
    plt.tight_layout()
    plt.show()

    # N_agua vs Impacto
    plt.figure(figsize=(8.5, 4.8))
    plt.scatter(work["N_agua_NO3N"], work["Impacto_fuzzy"])
    plt.title("Relación N_agua_NO3N vs Impacto_fuzzy", weight="bold")
    plt.xlabel("N_agua_NO3N (mg/L)")
    plt.ylabel("Impacto_fuzzy (0–100)")
    plt.grid(alpha=0.25)
    plt.tight_layout()
    plt.show()

    # MO vs Impacto
    plt.figure(figsize=(8.5, 4.8))
    plt.scatter(work["MO_suelo"], work["Impacto_fuzzy"])
    plt.title("Relación MO_suelo vs Impacto_fuzzy", weight="bold")
    plt.xlabel("MO_suelo (%)")
    plt.ylabel("Impacto_fuzzy (0–100)")
    plt.grid(alpha=0.25)
    plt.tight_layout()
    plt.show()

    # Barras por categoría
    counts = work["Riesgo_fuzzy"].value_counts().reindex(["Bajo", "Medio", "Alto"], fill_value=0)
    plt.figure(figsize=(6.8, 4.6))
    bars = plt.bar(counts.index, counts.values)
    plt.title("Conteo por categoría de Riesgo_fuzzy", weight="bold")
    plt.xlabel("Categoría")
    plt.ylabel("Número de registros")
    plt.grid(axis="y", alpha=0.25)
    for b in bars:
        y = b.get_height()
        plt.text(b.get_x() + b.get_width()/2, y + 0.1, f"{int(y)}", ha="center", va="bottom")
    plt.tight_layout()
    plt.show()


# ------------------------------------------------------------
# EJECUCIÓN (descomentar según necesidad)
# ------------------------------------------------------------
if __name__ == "__main__":

    # (1) Figuras del sistema conceptual (membresías)
    # plot_memberships()

    # (2) Superficie conceptual N vs MO (con DA y CE constantes)
    # plot_surface_N_MO(da_const=1.35, ce_const=900, n_points=45)

    # (3) Evaluación sobre datos reales USCO
    work = run_usco_dataset(USCO_URL)

    # Resumen (útil para redacción, no para pegar como output en anexos)
    print("Registros usados:", work.shape[0])
    print(work["Impacto_fuzzy"].describe())
    print(work["Riesgo_fuzzy"].value_counts())

    # Gráficas reproducibles
    plot_usco_results(work)

    # Export para Anexo E (resultados completos) y tabla de críticos
    work.to_csv("resultados_fuzzy_usco_completos.csv", index=False)
    work.sort_values("Impacto_fuzzy", ascending=False).head(10).to_csv("top10_criticos_usco.csv", index=False)

    print("✅ Exportados: resultados_fuzzy_usco_completos.csv y top10_criticos_usco.csv")
